# file set-up
1. import necessary libraries  
2. title directory (where data files are stored)  
3. create empty dataframe  
4. create list of languages
    will return to this---currently it doesn't actually serve a purpose, as I was unable to get the languages through here.  
    will either update this file or change spider to sort by language.  

In [1]:
from bs4 import BeautifulSoup as beau
import pandas as pd
import itertools
import pickle
import nltk
import os
import re

In [ ]:
directory = 'natfinder/natfinder/scraped-works'
fics_df = pd.DataFrame(columns=['L1', 'author', 'title', 'tags', 'summary', 'notes', 'endnotes', 'work', 'rating', 'warnings', 'category', 'fandom', 'ships', 'charas', 'freeform'])
langs = ['afrikaans', 'albanian', 'amharic', 'anii', 'arabic', 'araona', 'armenian', 'assamese', 'aymara', 'ayoreo', 'azerbaijan', 'balanta', 'bambara', 'bariba', 'basque', 'bassari', 'baure', 'bedik', 'belarusian', 'bengali', 'berber', 'biali', 'bislama', 'boko', 'bomu', 'bosnian', 'bozo', 'buduma', 'bulgarian', 'burmese', 'canichana', 'cantonese', 'carolinian', 'catalan', 'cayubaba', 'chacobo', 'chamorro', 'chichewa', 'chinese', 'chirbawe', 'comorian', 'corsican', 'creole', 'croation', 'czech', 'dagaare', 'dagbani', 'dangme', 'danish', 'dari', 'daroese', 'dendi', 'dhivehi', 'dioula', 'dogon', 'dutch', 'estonian', 'fante', 'figian', 'filipino', 'finnish', 'foodo', 'formosan', 'french', 'fula', 'gaelic', 'gbe', 'georgian', 'german', 'gonja', 'gourmanche', 'greek', 'guarani', 'guarayu', 'gujarati', 'hakka', 'hassaniya', 'hausa', 'hebrew', 'hindi', 'hiri', 'hokkien', 'hungarian', 'icelandic', 'igbo', 'indonesian', 'irish', 'italian', 'itene', 'itonama', 'japanese', 'javanese', 'jerriais', 'jola', 'kabye', 'kalanga', 'kallawaya', 'kannada', 'kanuri', 'kasem', 'kazakh', 'khmer', 'kinyarwanda', 'kirundi', 'kissi', 'koisan', 'korean', 'kpelle', 'kurdish', 'kyrgyz', 'lao', 'latvian', 'leco', 'lithuanian', 'lukpa', 'luzembourgish', 'macedonian', 'malagasy', 'malay', 'malayalam', 'malinke', 'maltese', 'mamara', 'mandarin', 'manding', 'mandinka', 'mandjak', 'manipuri', 'mankanya', 'maori', 'marathi', 'marshallese', 'mbelime', 'meitei', 'mongolian', 'montenegrin', 'moseten', 'mossi', 'motu', 'movima', 'moxos', 'nambya', 'nateni', 'nauruan', 'ndau', 'ndebele', 'nepali', 'norwegian', 'nzema', 'oniyan', 'oriya', 'oromo', 'ossetian', 'pakawara', 'palauan', 'papiamento', 'pashto', 'persian', 'pisin', 'polish', 'portuguese', 'punjabi', 'puquina', 'quechua', 'romanian', 'romansh', 'russian', 'safen', 'sango', 'scots', 'scottish', 'sena', 'serbian', 'serer', 'sewdish', 'shinhala', 'shona', 'siriono', 'slovak', 'slovene', 'somali', 'soninke', 'sonsorolese', 'sotho', 'spanish', 'susu', 'swahili', 'swati', 'syenara', 'tacana', 'tagalog', 'tajik', 'tamasheq', 'tamil', 'tammari', 'tapiete', 'tasawaq', 'tebu', 'telugu', 'tetum', 'thai', 'tigrinya', 'tobian', 'toma', 'tonga', 'tongan', 'toromono', 'tsonga', 'tswana', 'turkish', 'turkmen', 'tuvaluan', 'twi', 'ukrainian', 'urdu', 'uzbek', 'venda', 'vietnamese', 'waama', 'wamey', 'weenhayek', 'welsh', 'wolof', 'xhosa', 'yaminawa', 'yobe', 'yom', 'yoruba', 'yuki', 'yuracare', 'zarma', 'zulu']

# def: extract_info(soup)
1. try/except in case data is missing from file (ie; fics written by anonymous users will not return an author value)  
2. gather basic info (author, title, tags, summary, notes, endnotes, work)  
    tag info (rating, warning, category, fandom, (relation)ships, characters, freeform)  
    stat info (publication date, status, word count, chapters, comments, kudos, hits, bookmarks)
3. add to `fics_df`

In [3]:
def extract_info(soup):
    # basic info
    try:
        author = soup.find(rel='author').text
    except:
        author = 'NaN'
    try:
        title = soup.find(class_='title heading').text
    except:
        title = 'NaN'
    try:
        tags = [t.text for t in soup.find_all(class_='tag')]
    except:
        tags = 'NaN'
    try:
        summary = soup.find('div', class_='summary module').text
    except:
        summary = 'NaN'
    try:
        notes = soup.find('div', class_='notes module').text
    except:
        notes = 'NaN'
    try:
        endnotes = soup.find('div', class_='end notes module').text
    except:
        endnotes = 'NaN'
    try:
        work = soup.text
    except:
        work = 'NaN'
    # tag info
    try:
        rating = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='rating tags')]
    except:
        rating = 'ERROR'
    try:
        warnings = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='warning tags')]
    except:
        warnings = 'ERROR'
    try:
        categories = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='category tags')]
    except:
        categories = 'ERROR'
    try:
        fandoms = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='fandom tags')]
    except:
        fandoms = 'ERROR'
    try:
        ships = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='relationship tags')]
    except:
        ships = 'NaN'
    try:
        charas = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='character tags')]
    except:
        charas = 'NaN'
    try:
        freeform = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='freeform tags')]
    except:
        freeform = 'NaN'
    temp = pd.DataFrame({'author':[author], 'title':[title],'tags':[tags], 'summary':[summary], 'notes':[notes], 'endnotes':[endnotes], 'work':[work], 'rating':[rating], 'warnings':[warnings], 'category':[categories], 'fandom':[fandoms], 'ships':[ships], 'charas':[charas], 'freeform':[freeform]})
    return temp

# parsing through files
1. gather files  
2. create bs4 soup for files
3. use `extract_info(soup)` to add to fics_df

In [4]:
files = []
for f in os.scandir(directory):
    if f.name.endswith('.html'):
        files.append(f.name)
for f in files:
    soup = beau(open(directory + '/' + f).read(), 'html.parser')
    fics_df = pd.concat([fics_df, extract_info(soup)], ignore_index=True)

4. clean up data

In [5]:
fics_df['work'] = fics_df.work.map(lambda x : re.sub(r'(?s).*Work Text:', '', re.sub(r'(?s)\xa0\n+.*', '', re.sub(r'\s', ' ', x))))

5. pickle dataframe as `fics_df_scraped.pkl`

In [ ]:
#with open('fics_df_scraped.pkl', 'wb') as file:
    #pickle.dump(fics_df, file)

6. come back and find L1 of author

In [139]:
with open('fics_df_scraped.pkl', 'rb') as file:
    fics_df = pickle.load(file)

In [140]:
def find_lang(fic):
    for l in langs:
        if l in nltk.word_tokenize(fic.lower()):
            return l

In [141]:
def find_lang_tags(fic):
    for l in langs:
        for x in fic:
            if l in nltk.word_tokenize(x.lower()):
                return l

In [163]:
fics_df['L1s'] = fics_df.summary.map(lambda x : find_lang(x))

In [162]:
fics_df['L1n'] = fics_df.notes.map(lambda x : find_lang(x))

In [161]:
fics_df['L1e'] = fics_df.endnotes.map(lambda x : find_lang(x))

In [160]:
fics_df['L1t'] = fics_df.tags.map(lambda x : find_lang_tags(x))

In [149]:
fics_df['L1'] = fics_df[['L1s', 'L1n', 'L1e', 'L1t']].bfill(axis=1).iloc[:, 0]


In [152]:
fics_df.drop(columns=['L1s', 'L1n', 'L1e', 'L1t'], inplace=True)

In [155]:
fics_df_lang = fics_df[fics_df['L1'].notna()]

In [157]:
fics_df_mys = fics_df[fics_df['L1'].isna()]

In [159]:
with open('fics_df_scraped_lang.pkl', 'wb') as file:
    pickle.dump(fics_df_lang, file)
with open('fics_df_scraped_mys.pkl', 'wb') as file:
    pickle.dump(fics_df_mys, file)